In [7]:
import gradio as gr
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Define LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama3-70b-8192"
)

# === Shared State ===
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation history"]
    city: str
    interests: List[str]
    days: int
    itinerary: str
    image_urls: List[str]

# === Agent 1: Interest Interpreter ===
def interest_agent(state: PlannerState) -> PlannerState:
    parsed_interests = [i.strip() for i in state["messages"][-1].content.split(',')]
    state["interests"] = parsed_interests
    return state

# === Agent 2: Itinerary Builder ===
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a travel assistant. Create a {days}-day itinerary for {city} based on these interests: {interests}. Use bullet points."),
    ("human", "Plan my trip.")
])

def itinerary_agent(state: PlannerState) -> PlannerState:
    msg = itinerary_prompt.format_messages(
        city=state["city"],
        interests=", ".join(state["interests"]),
        days=state["days"]
    )
    response = llm.invoke(msg)
    state["itinerary"] = response.content
    state["messages"].append(AIMessage(content=response.content))
    return state

# === Agent 3: Image Search Agent ===
def image_agent(state: PlannerState) -> PlannerState:
    def get_image_url(query):
      return f"https://picsum.photos/seed/{query}/600/400"

    state["image_urls"] = [get_image_url(i) for i in state["interests"]]
    return state

# === Graph Setup ===
builder = StateGraph(PlannerState)
builder.add_node("InterestInterpreter", interest_agent)
builder.add_node("ItineraryBuilder", itinerary_agent)
builder.add_node("ImageSearch", image_agent)

builder.set_entry_point("InterestInterpreter")
builder.add_edge("InterestInterpreter", "ItineraryBuilder")
builder.add_edge("ItineraryBuilder", "ImageSearch")
builder.add_edge("ImageSearch", END)

app_graph = builder.compile()

# === Main Function ===
def travel_planner(city: str, interests: str, days: int):
    initial_state: PlannerState = {
        "messages": [HumanMessage(content=interests)],
        "city": city,
        "interests": [],
        "days": days,
        "itinerary": "",
        "image_urls": [],
    }

    final_state = app_graph.invoke(initial_state)

    image_html = "".join([
    f"<div style='display:inline-block;margin:10px;text-align:center'>"
    f"<img src='{url}' width='250' height='160' style='border-radius:10px;box-shadow:0 0 10px rgba(0,0,0,0.3);'/><br>"
    f"<p style='font-size:14px'>{interest}</p>"
    f"</div>"
    for interest, url in zip(final_state["interests"], final_state["image_urls"])
])

    return f"### Your {days}-Day Itinerary for {city}\n\n{final_state['itinerary']}", image_html

# === Gradio Interface ===
interface = gr.Interface(
    fn=travel_planner,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="Enter the city for your trip"),
        gr.Textbox(label="Enter your interests (comma-separated)"),
        gr.Slider(1, 5, step=1, label="Number of Days")
    ],
    outputs=[
        gr.Markdown(label="Generated Itinerary"),
        gr.HTML(label="Visual Suggestions")
    ],
    title="🌍 Multi-Agent AI Travel Planner",
    description="An intelligent travel planner using multiple agents to understand interests, generate itineraries, and suggest images."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02e8b5875a765810b0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
# Multi-Agent AI Travel Planner using LangGraph

from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# === Setup LLM ===
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama3-70b-8192"
)

# === Shared State ===
class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "Conversation history"]
    city: str
    interests: List[str]
    days: int
    itinerary: str
    image_urls: List[str]
    budget: str
    food: str
    transport: str

# === Agent 1: Interest Interpreter ===
def interest_agent(state: PlannerState) -> PlannerState:
    parsed_interests = [i.strip() for i in state["messages"][-1].content.split(',')]
    state["interests"] = parsed_interests
    return state

# === Agent 2: Itinerary Builder ===
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "Create a {days}-day itinerary for {city} based on these interests: {interests}. Use bullet points."),
    ("human", "Plan my trip.")
])

def itinerary_agent(state: PlannerState) -> PlannerState:
    msg = itinerary_prompt.format_messages(
        city=state["city"],
        interests=", ".join(state["interests"]),
        days=state["days"]
    )
    response = llm.invoke(msg)
    state["itinerary"] = response.content
    state["messages"].append(AIMessage(content=response.content))
    return state

# === Agent 3: Image Search ===
def image_agent(state: PlannerState) -> PlannerState:
    def get_image_url(query):
        return f"https://picsum.photos/seed/{query}/600/400"
    state["image_urls"] = [get_image_url(i) for i in state["interests"]]
    return state

# === Agent 4: Budget Estimator ===
budget_prompt = ChatPromptTemplate.from_messages([
    ("system", "Estimate a budget for a {days}-day trip to {city} with interests: {interests}. Include travel, food, stay, and fun."),
    ("human", "How much should I plan to spend?")
])

def budget_agent(state: PlannerState) -> PlannerState:
    msg = budget_prompt.format_messages(
        city=state["city"],
        interests=", ".join(state["interests"]),
        days=state["days"]
    )
    response = llm.invoke(msg)
    state["budget"] = response.content
    return state

# === Agent 5: Food Advisor ===
food_prompt = ChatPromptTemplate.from_messages([
    ("system", "Suggest must-try foods and popular restaurants in {city} for someone interested in {interests}."),
    ("human", "What should I eat?")
])

def food_agent(state: PlannerState) -> PlannerState:
    msg = food_prompt.format_messages(
        city=state["city"],
        interests=", ".join(state["interests"])
    )
    response = llm.invoke(msg)
    state["food"] = response.content
    return state

# === Agent 6: Local Transport Advisor ===
transport_prompt = ChatPromptTemplate.from_messages([
    ("system", "List best local transport options in {city} for tourists: metro, taxi, auto, bike rentals, etc."),
    ("human", "How should I travel locally?")
])

def transport_agent(state: PlannerState) -> PlannerState:
    msg = transport_prompt.format_messages(city=state["city"])
    response = llm.invoke(msg)
    state["transport"] = response.content
    return state

# === Define Graph ===
builder = StateGraph(PlannerState)

builder.add_node("InterestInterpreter", interest_agent)
builder.add_node("ItineraryBuilder", itinerary_agent)
builder.add_node("ImageSearch", image_agent)
builder.add_node("BudgetEstimator", budget_agent)
builder.add_node("FoodAdvisor", food_agent)
builder.add_node("LocalTransport", transport_agent)

builder.set_entry_point("InterestInterpreter")
builder.add_edge("InterestInterpreter", "ItineraryBuilder")
builder.add_edge("ItineraryBuilder", "ImageSearch")
builder.add_edge("ImageSearch", "BudgetEstimator")
builder.add_edge("BudgetEstimator", "FoodAdvisor")
builder.add_edge("FoodAdvisor", "LocalTransport")
builder.add_edge("LocalTransport", END)

app_graph = builder.compile()

# === Inference Function ===
def run_travel_planner(city: str, interests: str, days: int):
    state: PlannerState = {
        "messages": [HumanMessage(content=interests)],
        "city": city,
        "interests": [],
        "days": days,
        "itinerary": "",
        "image_urls": [],
        "budget": "",
        "food": "",
        "transport": ""
    }

    result = app_graph.invoke(state)

    summary = f"""
### {days}-Day Itinerary for {city}

{result['itinerary']}

---

### Estimated Budget
{result['budget']}

### Local Transport Options
{result['transport']}

### Must-Try Foods & Restaurants
{result['food']}
"""

    image_html = "".join([
        f"<div style='display:inline-block;margin:10px;text-align:center'>"
        f"<img src='{url}' width='250' height='160' style='border-radius:10px;'/><br>"
        f"<p style='font-size:14px'>{interest}</p>"
        f"</div>"
        for interest, url in zip(result['interests'], result['image_urls'])
    ])

    return summary, image_html
import gradio as gr

interface = gr.Interface(
    fn=run_travel_planner,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="City", placeholder="e.g., Goa"),
        gr.Textbox(label="Interests (comma-separated)", placeholder="e.g., beaches, food, parties"),
        gr.Slider(1, 10, value=3, step=1, label="Number of Days")
    ],
    outputs=[
        gr.Markdown(label="Trip Summary"),
        gr.HTML(label="Visual Suggestions")
    ],
    title="🌍 Multi-Agent AI Travel Planner",
    description="Get a personalized itinerary, budget, food advice, and travel tips for your next trip!"
)

interface.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e8558e6cfee61f4d17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
result, images = run_travel_planner("Goa", "beach, nightlife, food", 3)
from IPython.display import display, Markdown, HTML

display(Markdown(result))
display(HTML(images))



### 3-Day Itinerary for Goa

Here's a 3-day itinerary for Goa based on your interests in beach, nightlife, and food:

**Day 1: Beaches and Local Cuisine**

* Morning: Start with a relaxing morning at Palolem Beach, known for its picturesque views and calm waters. You can take a leisurely walk along the shore, or try your hand at water sports like kayaking or paddleboarding.
* Lunch: Head to The Fish Factory at Palolem, which serves fresh seafood and local delicacies like fish curry and rice.
* Afternoon: Visit the nearby Patnem Beach, which is quieter and more secluded than Palolem. You can relax and soak up the sun here.
* Evening: For dinner, try out some local Goan cuisine at Huts and Shacks, a popular eatery at Palolem. Don't miss their thali, which includes a variety of local dishes.
* Night: End the day with a relaxing evening at Palolem's Silent Noise Party, where you can groove to music while wearing wireless headphones.

**Day 2: North Goa and Nightlife

* Morning: Head to North Goa and visit the popular Baga Beach, known for its vibrant atmosphere and water sports.
* Lunch: Grab a bite at one of the many shacks along the beach, like Britto's or Larios. Don't miss their seafood and local snacks like pani puri and bhel puri.
* Afternoon: Visit the nearby Anjuna Beach, which is famous for its flea market and vibrant atmosphere.
* Evening: For dinner, try out some international cuisine at Hill Top, a popular restaurant in Vagator. They serve a range of dishes, from Italian to Indian.
* Night: Experience Goa's nightlife at Club Cubana, a popular club in Arpora. Dance the night away with friends!

**Day 3: Foodie Delights and Beach Hopping**

* Morning: Start the day with a visit to the famous Calangute Beach, which is bustling with tourists and locals alike.
* Lunch: Head to the popular Infantaria Bakery and Cafe, known for its mouth-watering desserts and sandwiches.
* Afternoon: Visit the nearby Candolim Beach, which is quieter and more secluded than Calangute. You can relax and soak up the sun here.
* Evening: For dinner, try out some gourmet cuisine at Gunpowder, a popular restaurant in Assagao. They offer a range of dishes, including local specialties like fish curry and rice.
* Night: End the day with a relaxing evening at a beach shack, watching the stars and listening to the sound of the waves.

This itinerary should give you a good mix of beach time, local cuisine, and nightlife. Enjoy your trip to Goa!

---

### Estimated Budget
A 3-day trip to Goa can vary greatly depending on your travel style, accommodation choices, and activities. Here's a rough estimate of a budget for a 3-day trip to Goa:**

**Travel:**

* Round-trip flights from a major Indian city (e.g., Delhi, Mumbai, Bengaluru): ₹8,000 - ₹15,000 (approximately $110-$200 USD)
* Train or bus tickets: ₹2,000 - ₹5,000 (approximately $28-$70 USD)

**Accommodation:**

* Budget-friendly options: ₹1,500 - ₹3,000 (approximately $20-$40 USD) per night for a hostel or guesthouse
* Mid-range options: ₹3,000 - ₹6,000 (approximately $40-$80 USD) per night for a hotel or resort
* Luxury options: ₹8,000 - ₹15,000 (approximately $110-$200 USD) per night for a luxury resort or villa

**Food:**

* Street food and local eateries: ₹200 - ₹500 (approximately $2.80-$7 USD) per meal
* Mid-range restaurants: ₹500 - ₹1,000 (approximately $7-$14 USD) per meal
* Fine dining: ₹1,500 - ₹3,000 (approximately $20-$40 USD) per meal

**Fun and activities:**

* Beach activities (e.g., water sports, sunbeds): ₹500 - ₹1,000 (approximately $7-$14 USD) per day
* Nightlife (e.g., club entry, drinks): ₹1,000 - ₹2,000 (approximately $14-$28 USD) per night
* Sightseeing (e.g., Fort Aguada, Dudhsagar Falls): ₹500 - ₹1,000 (approximately $7-$14 USD) per person

**Total estimated budget:**

* Budget-friendly: ₹20,000 - ₹35,000 (approximately $280-$490 USD) for 3 days
* Mid-range: ₹35,000 - ₹60,000 (approximately $490-$840 USD) for 3 days
* Luxury: ₹60,000 - ₹1,20,000 (approximately $840-$1,680 USD) for 3 days

Keep in mind that these are rough estimates, and your actual expenses may vary depending on your specific travel plans and preferences. Additionally, prices may be higher during peak season (December to February).

Remember to also budget for any additional expenses, such as souvenirs, transportation within Goa, and any additional activities or excursions you might want to do.

### Local Transport Options
Goa! There are several local transport options in Goa that are convenient, affordable, and fun. Here are some of the best local transport options in Goa:

1. **Two-Wheelers (Bikes and Scooters) on Hire**: This is the most popular mode of transport in Goa. You can rent a bike or scooter for a day (₹300-₹500) or longer periods. Many rental shops are available near popular beaches and tourist areas.
2. **Taxis**: Taxis are widely available, but they can be more expensive than two-wheelers. You can hire a taxi for a day (₹800-₹1,200) or pay by the kilometer (₹8-₹12 per km).
3. **Auto Rickshaws**: Autos are a good option for short distances. They are widely available, but be prepared to negotiate the fare.
4. **Local Buses**: The Kadamba Transport Corporation (KTC) operates bus services across Goa. They are an affordable option (₹10-₹50) but may not be as frequent as other modes of transport.
5. **Ferries**: If you want to explore the rivers and backwaters of Goa, ferries are a great option. They operate between various points, including Panaji, Old Goa, and Vasco da Gama.
6. **Rent a Car**: If you prefer to drive a car, there are many rental services in Goa. This option is more expensive than two-wheelers, but convenient for groups or families.
7. **App-based Taxi Services**: Ola and other app-based taxi services are available in Goa, but they may not be as frequent as in larger cities.
8. **Be sure to wear a helmet**: If you rent a two-wheeler, always wear a helmet to avoid fines and ensure your safety.

**Before you rent a vehicle, make sure to:**

* Check the vehicle's condition
* Negotiate the fare or rental charges
* Ensure you have a valid driving license
* Wear a helmet (if renting a two-wheeler)

**Tips:**

* Drive safely and defensively, as Goa's roads can be chaotic.
* Be aware of pedestrians, cyclists, and other vehicles.
* Avoid driving at night, if possible.

Now, get ready to explore the beauty of Goa!

### Must-Try Foods & Restaurants
Goa is a foodie's paradise! You'll discover a unique blend of Konkani, Portuguese, and Indian cuisines. Here are some must-try foods in Goa:

1. **Seafood**: Being a coastal state, Goa offers an array of fresh seafood dishes. Try fish curry, fish fry, prawns, lobsters, and crab.
2. **Prawn Curry**: A classic Goan dish made with prawns, coconut milk, and spices.
3. **Fish Thali**: A traditional meal consisting of fish curry, rice, and vegetables.
4. **Beef Chilly**: A spicy dish made with beef, onions, and chilies.
5. **Chicken Cafreal**: A Portuguese-inspired dish made with marinated chicken, served with rice and potatoes.
6. **Vindaloo**: A popular curry dish made with meat (usually pork or chicken), vinegar, and spices.
7. **Xacuti**: A rich and flavorful curry made with chicken or beef, and a blend of spices.
8. **local snacks**: Try the famous Goan pao (bread), accompanied by bhaji (fritters) or misal (a spicy curry).
9. **Feni**: A local spirit made from cashew or palm sap. Try it with lime and soda for a unique experience.

Now, let's explore some popular restaurants in Goa:

**Beachside Eats:**

1. **La Plage** (Ashwem Beach): Savor French-Goan cuisine with a stunning beach view.
2. **Gunpowder** (Assagao): Enjoy South Indian and Konkani dishes in a rustic setting.

**Nightlife Hotspots:**

1. **Café Mambo** (Baga Beach): A popular spot for seafood, cocktails, and live music.
2. **Sinq** (Candolim): A trendy restaurant and bar serving international cuisine.

**Local Favorites:**

1. **Hill Top** (Vagator): A laid-back restaurant serving Goan and international dishes.
2. **Infantaria** (Calangute): A popular bakery and café serving delicious pastries and sandwiches.
3. **Lila Café** (Baga): A cozy café serving healthy food, smoothies, and desserts**.

Remember to also try some local street food and markets, like the **Mapusa Market**, to experience the authentic flavors of Goa.

Enjoy your food adventure in this beautiful beach state!
